In [39]:
import tensorflow as tf
import numpy as np
from math import exp
from numpy.random import binomial
from random import shuffle

nx = 5
ny = 5
nz = 5
N = nx*ny*nz
print(tf.version,'\n', tf.config.list_physical_devices('GPU'))

<module 'tensorflow._api.v2.version' from 'C:\\Users\\aseer\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'> 
 []


In [40]:
# Initializing the neurons
# LSM_neurons = tf.zeros((nx,ny,nz),dtype=tf.float64)

# print(LSM_neurons)

In [41]:
# Storing the IDs of the neurons
LSM_ID = np.zeros((nx,ny,nz),dtype=np.int64)
l = 0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            LSM_ID[i,j,k] = l
            l = l + 1;

LSM_ID = tf.convert_to_tensor(LSM_ID,dtype=tf.int64)
# print(LSM_ID)

In [48]:
def ID_to_ind(nx,ny,nz,ID):
    x = int(ID/(ny*nz))
    y = int( (ID-(ny*nz)*x) / nz)
    z = int(ID%nz)
    return [x, y, z]

In [55]:
# Storing the synapse connections
k_prob = 1
r_sq = 1**2

W_init = 100 # initial weight

N_in = int(N*0.8)
neuron_type = [ int(i<N_in) for i in range(N)]
shuffle(neuron_type) # 1 for excitatory, 0 for inhibitory

synapes = [dict() for i in range(N)]    # an array of dictonaries which store the location of neuron, type of neuron, and the IDs and weights of the synapes of the neurons it is connected to

for l in range(N):
    loc = ID_to_ind(nx,ny,nz,l)
    n_type = neuron_type[l]
    cons = []
    weights = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                dist_sq = (loc[0]-i)**2 + (loc[1]-j)**2 + (loc[2]-k)**2
                probability = k_prob* exp(-1*dist_sq/r_sq)
                check = binomial(1,probability)
                if check == 1:
                    cons.append(int(LSM_ID[i,j,k]))
                    weights.append(W_init)
    
    synapes[l] = {"Location":loc, "Neuron_type":n_type, "connections":cons, "weigths":weights}


In [ ]:
from brian2 import *

#### LIF Neuron
Converted from MATLAB.  Modified version of LIF solver for HW1, with N neurons and refractory period added, solves just a single timestep 

In [12]:
params = {'C':300, 'g_L':30, 'E_L':-70, 'V_T':20, 'R_p':2}
C, g_L, E_L, V_T, R_p = params.values()
def LIF(V_neuron_prev,I_input_prev,I_input_next,nx,ny,nz,h,index_next,index_prev_spike, params):
    C, g_L, E_L, V_T, R_p = params.values()
    R_p_ind = tf.math.ceil(R_p/h)
    
    V_neuron_next = tf.math.scalar_mul(E_L,tf.ones((nx,ny,nz), dtype='float'))
    Spike_next = tf.zeros((nx,ny,nz), dtype='float')
    
    k1 = (1/C)*(-g_L*(V_neuron_prev-E_L)+I_input_prev)
    V_temp = V_neuron_prev + k1*h/2
    I_temp = I_input_prev/2 + I_input_next/2
    k2 = (1/C)*(-g_L*(V_temp-E_L)+I_temp)
    V_temp = V_neuron_prev + k2*h
    
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if index_next-index_prev_spike[i,j,k] < R_p_ind:
                    V_neuron_next[i,j,k] = E_L
                elif V_temp[i,j,k] < V_T:
                    V_neuron_next[i,j,k] = V_temp[i,j,k]
                else:
                    Spike_next[i,j,k] = 1
                    V_neuron_next[i,j,k] = V_temp[i,j,k]
    
    return V_neuron_next, Spike_next
    
            

#### Simplified model in paper(IEEE ZHANG et al)
Effect of Synaptic models **Section V(A), eqns -- 19, 20, 21**

In [11]:
from exceptions import ValueError
global vrest, vth, t_refrac
vrest, vth, t_refrac = -70, 20, 2

In [14]:
def syn_res(syn_string, time_ind, spike_timing_mat, h, type_syn=None):
    
    if syn_string == "static":
        result_mat = [1 if time_ind == a_spike else 0 for a_spike in spike_timing_mat]
        return result_mat
    elif syn_string == "first-order":
        tau_s = 4 * h # Given in paper
        result_mat = (1/tau_s) * (np.exp(- (time_ind - spike_timing_mat)/tau_s) 
        * np.heaviside(time_ind - spike_timing_mat))
        return result_mat
    elif syn_string == "second-order":
        if type_syn is None:
            raise ValueError('type required')
        elif type_syn == 'Exc':
            tau_s1, tau_s2 = 4, 8
        elif type_syn == 'Inh':
            tau_s1, tau_s2 = 4, 2
        else:
            raise ValueError('type invalid')
        result_mat = ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s1)
                      * np.heaviside(time_ind - spike_timing_mat)
                      - ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s2)
                      * np.heaviside(time_ind - spike_timing_mat)))
        
        return result_mat
            

Models for implementing proposed learning rule **Section IV(c), eqns -- 12, 13, 14**

In [13]:
def LIF_rsp_one(neuron_id,Weight_mat, Vm_prev, FanIn_list, syn_string, 
                tau_m, spikeTime_mat, Delay_mat, it_prev, h, time_ind, **kwargs):
    global vrest, vth
    spike_happened = 0
    V_temp = Vm_prev  - Vm_prev/tau_m + it_prev
    for i in FanIn_list:
        V_temp += Weight_mat[i] * syn_res(syn_string, time_ind, spikeTime_mat[:, i] + Delay_mat[i], h) 
    if time_ind - spikeTime_mat[neuron_id][-1] <= tf.math.ceil(t_refrac/h):
        V_temp = vrest
    elif V_temp > vth:
        spikeTime_mat[time_ind, neuron_id] = 1
        V_temp = vrest
    else:
        pass
    return V_temp
    
    

Weight learning by calculation of Digitized Calcium concentration updation, **Equations 15 , 16 in IEEE paper**<br>
Incomplete, have to read section **III(A), III(B)**

In [ ]:
def Weight_learner(tau_c, last_conc, time_ind, this_neuron_spiked):
    new_conc = last_conc - last_conc/tau_c + np.sum(np.heaviside(time_ind - this_neuron_spiked))
    ## Need to implement the learning rule here 
    return new_conc
    

In [ ]:
def synaptic_current(nx,ny,nz,M,t,Spike,synapes):
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if Spike[i,j,k] == 1:
                    l = len(synapes[LSM_ID[i,j,k]]["connections"])
                    for m in range(l):
                        # add code here
                        asdf


Converted from **MATLAB assignment 3 Q2** neuron solver <br>

In [ ]:
def reservoir_solver(nx,ny,nz, synapes, M, h, I_app,threshold, **kwargs):
    global vrest, vth
    I_syn = tf.zeros((nx,ny,nz,M))
    I_total = tf.zeros((nx,ny,nz,M))
    V_neurons = vrest*tf.ones((nx,ny,nz,M))
    Spikes = tf.zeros((nx,ny,nz,M))
    Calcium_conc = tf.zeros((nx,ny,nz,M))
    weight_mat = tf.ones((nx,ny,nz,M))

    tau_m, syn_string, h = 32, "static", 0.5
    
    index_prev_spike = -1*(M)*tf.ones((nx,ny,nz))

    for t in range(1,M):
        I_total[:,:,:,t] = I_app[:,:,:,t] + I_syn[:,:,:,t]
        # for j in range(N):
            # V_neuron[j,i] = LIF_rsp_one(j, Weight_mat[j], V_neuron[j-1,i], Connect_mat[j], syn_string,
            #                            tau_m, Spikes, Delay_mat, I_total[:,i-1], h, time_ind=i,
            #                            Calcium_conc = Calcium_conc)
        V_neuron, Spike = LIF(V_neurons[:,:,:,t-1],I_total[:,:,:,t-1],I_total[:,:,:,t],nx,ny,nz,h,t,index_prev_spike, params)
        V_neurons[:,:,:,t] = V_neuron
        Spikes[:,:,:,t] = Spike
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    if Spike[i,j,k] == 1:
                        index_prev_spike[i,j,k] = i
                        # not storing spike time for now
        I_syn_new = tf.zeros((nx,ny,nz))
        for i in range(nx):
            for j in range(ny):
                for k in range(nz):
                    # write a separate function synaptic_current for synaptic current update
                    I_syn_new = synaptic_current(nx,ny,nz,M,t,Spikes,synapes)
                    
        I_syn = I_syn + I_syn_new
    
    return V_neurons, Spikes
    

In [ ]:
# LSM_ID
# tf.reshape(LSM_ID,(nx,ny,nz,1))